In [1]:
# Imports
# !pip install torchinfo
import torch
import torch.nn as nn
import torch.utils.data as Data
from torchvision import datasets, transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import numpy as np
import random
import Trainer_V06C1 as TP
import CompMethod as TP_Comp

In [ ]:
gpu = 4

In [2]:
datadir = '../Dataset/0307make07/'
print(os.listdir(datadir))

loaded
(7638, 2, 90, 100) (7638, 1, 128, 128)
6110 763 765
6110 763 765


In [ ]:
mydata = DS.MyDataset(name='comp',
                      csi_path = datadir + 'csi_f.npy',
                      img_path = datadir + 'c_img.npy',
                      img_size=(128, 128),
                       mmap_mode=None
                     )
mydata.load_data()
myloader = DS.DataSplitter(mydata, train_ratio=0.8)

train_loader, valid_loader, test_loader = myloader.split_loader()

In [3]:
# ----Don't use ToPILImage() or ToTensor()----
transform = transforms.Compose([
    transforms.Resize([240, 320]),
 ])


mydata_wi2vi = DS.MyDataset(name='comp',
                      csi_path = datadir + 'csi_f.npy',
                      img_path = datadir + 'r_img.npy',
                      img_size=(226, 128),
                            transform=transform,
                       mmap_mode=None
                     )
mydata_wi2vi.load_data()
myloader = DS.DataSplitter(mydata_wi2vi, train_ratio=0.8)

train_loader, valid_loader, test_loader = myloader.split_loader()

loaded
(7638, 2, 90, 100) (7638, 1, 128, 128)
6110 763 765
6110 763 765


In [ ]:
select_ind = np.random.choice(765, 8).tolist()

### Proposed

In [6]:
torch.cuda.set_device(gpu)
imgencoder = TP.ImageEncoder(latent_dim=16)
imgdecoder = TP.ImageDecoder(active_func=nn.Sigmoid(),latent_dim=16)
csiencoder = TP.CSIEncoder(out_length=32)

imgencoder.load_state_dict(torch.load('../saved/0827r2/0827r2_ImgEnV03b3FcTe100_Se400.pth'))
imgdecoder.load_state_dict(torch.load('../saved/0827r2/0827r2_ImgDeV03b3Te100_Se400.pth'))
csiencoder.load_state_dict(torch.load('../saved/0827r2/0827r2_Model_v03b2_CsiEn_LastTe100_Se400.pth'))

S_trainer_mask = TP.StudentTrainerMask(
                              name='PropStudent', networks=[csiencoder, imgencoder, imgdecoder],
                 lr=1e-4, epochs=10, cuda=gpu,
                 train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)

In [ ]:
S_trainer_mask.test(loader=test_loader)
S_trainer_mask.plot_test(select_num=8, autosave=True, notion='240304Bench')

In [8]:
np.save('../saved/240304Bench/prop.npy', S_trainer_mask.loss.loss['pred']['IMG'])

### AutoEncoder

In [7]:
torch.cuda.set_device(gpu)
ae = TP.AutoEncoder(latent_dim=16)
ae.load_state_dict(torch.load('../saved/Comp_ae.pth'))

trainer_ae = TP.CompTrainer(name='AE', mode='ae', mask=True,
                              networks=[ae],
                 lr=1e-4, epochs=10, cuda=gpu,
                 train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)

In [ ]:
trainer_ae.test(mode='test')
trainer_ae.plot_test(select_num=8, autosave=True, notion='240304Bench')

In [23]:
np.save('../saved/240304Bench/ae.npy', trainer_ae.loss.loss['pred']['LOSS'])

### VAE

In [ ]:
torch.cuda.set_device(gpu)
imgdecoder = TP.ImageDecoder(latent_dim=16)
csiencoder = TP.CSIEncoder(out_length=32)
trainer_vae = TP.CompTrainer(name='VAE', mode='vae', mask=True,
                              networks=[csiencoder, imgdecoder],
                 lr=1e-4, epochs=10, cuda=gpu,
                 train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)

In [ ]:
trainer_vae.test(mode='test')
trainer_vae.plot_test(select_ind=[0, 125, 250, 375, 500, 625, 750, 875],select_num=8, autosave=True, notion='240304Bench')

### TS_AE

In [8]:
torch.cuda.set_device(gpu)
imgencoder = TP.ImageEncoder(latent_dim=16)
imgdecoder = TP.ImageDecoder(latent_dim=16)
csiencoder = TP.CSIEncoder(out_length=16)

trainer_ae_s = TP.CompTrainerStudent(
                              name='TSAE_Student', mask=True,
    networks=[csiencoder, imgencoder, imgdecoder],
                 lr=1e-4, epochs=10, cuda=gpu,
                 train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)

In [ ]:
trainer_ae_s.test(loader=test_loader)
trainer_ae_s.plot_test(select_num=8, autosave=True, notion='240304Bench')

In [24]:
np.save('../saved/0829ts.npy', trainer_ts.test_loss['s']['IMG'])

### Wi2Vi

In [9]:
torch.cuda.set_device(gpu)
wi2vi = TP.Wi2Vi()
wi2vi.load_state_dict(torch.load('../saved/0829r1/0829r1_Wi2Vi_e200.pth'))

trainer_wi2vi = TP.CompTrainer(name='Wi2Vi', mask=True,
                              networks=[wi2vi],
                 lr=1e-4, epochs=10, cuda=gpu,
                 train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader)

In [10]:
trainer_wi2vi.test(mode='test')
trainer_wi2vi.plot_test(select_ind=[0, 125,250,375,500,625,750,875],select_num=8, autosave=True, notion='240304Bench')

Student: test=612/765, loss=0.11455684155225754
Test finished. Average loss={'LOSS': 0.5105084423024475, 'STRA': 1.6379321341908055, 'DIST': 0.027326828888708764, 'IMG': 168.13128442452623}


In [25]:
np.save('../saved/0829wi2vi.npy', trainer_wi2vi.test_loss['LOSS'])